<a href="https://colab.research.google.com/github/natarajankrishna/natarajan/blob/main/sentiment_analysis_twitter_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import re
import time
import string
import warnings

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from nltk.classify import NaiveBayesClassifier
from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from textblob.np_extractors import ConllExtractor
import pickle
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.pipeline import make_pipeline
from nltk.tokenize import RegexpTokenizer


In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')
nltk.download('movie_reviews')
nltk.download('punkt')
nltk.download('conll2000')
nltk.download('brown')
stopwords = set(stopwords.words("english"))
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 2010.csv to 2010 (1).csv
Saving 2011.csv to 2011.csv
Saving 2012.csv to 2012.csv
Saving 2013.csv to 2013.csv
Saving 2014.csv to 2014.csv
Saving 2015.csv to 2015.csv
Saving 2016.csv to 2016.csv
Saving 2017.csv to 2017.csv
Saving 2018.csv to 2018.csv
Saving 2019.csv to 2019.csv
Saving 2020.csv to 2020.csv
Saving 2021.csv to 2021.csv
Saving 2022.csv to 2022.csv


In [ ]:
df = pd.concat(map(pd.read_csv, ['2010.csv','2011.csv','2012.csv','2013.csv','2014.csv','2015.csv','2016.csv','2017.csv','2018.csv','2019.csv','2020.csv','2021.csv','2022.csv']))

In [ ]:
df.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,reply_to,retweet_date,translate,trans_src,trans_dest,time,mentions,replies_count,retweets_count,likes_count
0,0.0,15434727182,15434727182,1275676317000.0,2010-06-04 18:31:57,0,NaN,"Please ignore prior tweets, as that was someon...",en,[],...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.0,152153637639028736,152151847614943233,1325111228000.0,2011-12-28 22:27:08,0,NaN,@TheOnion So true :),en,[],...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,151809315026636800,151809315026636800,1325029135000.0,2011-12-27 23:38:55,0,NaN,If you ever wanted to know the *real* truth ab...,en,[],...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,151338939389706242,151338939389706242,1324916990000.0,2011-12-26 16:29:50,0,NaN,Walked around a neighborhood recently rebuilt ...,en,[],...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,151337237429239808,151337237429239808,1324916584000.0,2011-12-26 16:23:04,0,NaN,"It was Xmas, so we brought presents for the ki...",en,[],...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def fetch_sentiment_using_SIA(text):
    sid = SentimentIntensityAnalyzer()
    polarity_scores = sid.polarity_scores(text)
    return 'neg' if polarity_scores['neg'] > polarity_scores['pos'] else 'pos'

In [ ]:
sentiments_using_SIA = df.tweet.apply(lambda tweet: fetch_sentiment_using_SIA(tweet))
pd.DataFrame(sentiments_using_SIA.value_counts())

,tweet
pos,29713
neg,5165


In [ ]:
def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)
    
    return text 

In [ ]:
df['tidy_tweets'] = np.vectorize(remove_pattern)(df['tweet'], "@[\w]*: | *RT*")
df.head(10)

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,retweet_date,translate,trans_src,trans_dest,time,mentions,replies_count,retweets_count,likes_count,tidy_tweets
0,0.0,15434727182,15434727182,1275676317000.0,2010-06-04 18:31:57,0,NaN,"Please ignore prior tweets, as that was someon...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Please ignore prior tweets, as that was someon..."
0,0.0,152153637639028736,152151847614943233,1325111228000.0,2011-12-28 22:27:08,0,NaN,@TheOnion So true :),en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,@TheOnion So true :)
1,1.0,151809315026636800,151809315026636800,1325029135000.0,2011-12-27 23:38:55,0,NaN,If you ever wanted to know the *real* truth ab...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,If you ever wanted to know the *real* truth ab...
2,2.0,151338939389706242,151338939389706242,1324916990000.0,2011-12-26 16:29:50,0,NaN,Walked around a neighborhood recently rebuilt ...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walked around a neighborhood recently rebuilt ...
3,3.0,151337237429239808,151337237429239808,1324916584000.0,2011-12-26 16:23:04,0,NaN,"It was Xmas, so we brought presents for the ki...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"It was Xmas, so we brought presents for the ki..."
4,4.0,151327734843445249,151327734843445249,1324914318000.0,2011-12-26 15:45:18,0,NaN,"Met with UNICEF, Doctors Without Borders and A...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Met with UNICEF, Doctors Without Borders and A..."
5,5.0,151322293174419456,151322293174419456,1324913020000.0,2011-12-26 15:23:40,0,NaN,Just returned from a trip to Haiti. Covered a ...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Just returned from a trip to Haiti. Covered a ...
6,6.0,151317672217419777,151317672217419777,1324911919000.0,2011-12-26 15:05:19,0,NaN,Single character Tweets are the ulitmate exten...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single character Tweets are the ulitmate exten...
7,7.0,151151777662779392,151151777662779392,1324872366000.0,2011-12-26 04:06:06,0,NaN,I,und,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I
8,8.0,150390624552615937,150390624552615937,1324690893000.0,2011-12-24 01:41:33,0,NaN,The Russians are having some challenges with t...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Theussians are having some challenges with the...


In [ ]:
cleaned_tweets = []

for index, row in df.iterrows():
     words_without_links = [word for word in row.tidy_tweets.split() if 'http' not in word]
     cleaned_tweets.append(' '.join(words_without_links))

df['tidy_tweets'] = cleaned_tweets
df.head(10)

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,retweet_date,translate,trans_src,trans_dest,time,mentions,replies_count,retweets_count,likes_count,tidy_tweets
0,0.0,15434727182,15434727182,1275676317000.0,2010-06-04 18:31:57,0,NaN,"Please ignore prior tweets, as that was someon...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Please ignore prior tweets, as that was someon..."
0,0.0,152153637639028736,152151847614943233,1325111228000.0,2011-12-28 22:27:08,0,NaN,@TheOnion So true :),en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,@TheOnion So true :)
1,1.0,151809315026636800,151809315026636800,1325029135000.0,2011-12-27 23:38:55,0,NaN,If you ever wanted to know the *real* truth ab...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,If you ever wanted to know the *real* truth ab...
2,2.0,151338939389706242,151338939389706242,1324916990000.0,2011-12-26 16:29:50,0,NaN,Walked around a neighborhood recently rebuilt ...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walked around a neighborhood recently rebuilt ...
3,3.0,151337237429239808,151337237429239808,1324916584000.0,2011-12-26 16:23:04,0,NaN,"It was Xmas, so we brought presents for the ki...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"It was Xmas, so we brought presents for the ki..."
4,4.0,151327734843445249,151327734843445249,1324914318000.0,2011-12-26 15:45:18,0,NaN,"Met with UNICEF, Doctors Without Borders and A...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Met with UNICEF, Doctors Without Borders and A..."
5,5.0,151322293174419456,151322293174419456,1324913020000.0,2011-12-26 15:23:40,0,NaN,Just returned from a trip to Haiti. Covered a ...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Just returned from a trip to Haiti. Covered a ...
6,6.0,151317672217419777,151317672217419777,1324911919000.0,2011-12-26 15:05:19,0,NaN,Single character Tweets are the ulitmate exten...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single character Tweets are the ulitmate exten...
7,7.0,151151777662779392,151151777662779392,1324872366000.0,2011-12-26 04:06:06,0,NaN,I,und,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I
8,8.0,150390624552615937,150390624552615937,1324690893000.0,2011-12-24 01:41:33,0,NaN,The Russians are having some challenges with t...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Theussians are having some challenges with the...


In [ ]:
df = df[df['tidy_tweets']!='']
df.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,retweet_date,translate,trans_src,trans_dest,time,mentions,replies_count,retweets_count,likes_count,tidy_tweets
0,0.0,15434727182,15434727182,1275676317000.0,2010-06-04 18:31:57,0,NaN,"Please ignore prior tweets, as that was someon...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Please ignore prior tweets, as that was someon..."
0,0.0,152153637639028736,152151847614943233,1325111228000.0,2011-12-28 22:27:08,0,NaN,@TheOnion So true :),en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,@TheOnion So true :)
1,1.0,151809315026636800,151809315026636800,1325029135000.0,2011-12-27 23:38:55,0,NaN,If you ever wanted to know the *real* truth ab...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,If you ever wanted to know the *real* truth ab...
2,2.0,151338939389706242,151338939389706242,1324916990000.0,2011-12-26 16:29:50,0,NaN,Walked around a neighborhood recently rebuilt ...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walked around a neighborhood recently rebuilt ...
3,3.0,151337237429239808,151337237429239808,1324916584000.0,2011-12-26 16:23:04,0,NaN,"It was Xmas, so we brought presents for the ki...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"It was Xmas, so we brought presents for the ki..."


In [ ]:
df.drop_duplicates(subset=['tidy_tweets'], keep=False)
df.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,retweet_date,translate,trans_src,trans_dest,time,mentions,replies_count,retweets_count,likes_count,tidy_tweets
0,0.0,15434727182,15434727182,1275676317000.0,2010-06-04 18:31:57,0,NaN,"Please ignore prior tweets, as that was someon...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Please ignore prior tweets, as that was someon..."
0,0.0,152153637639028736,152151847614943233,1325111228000.0,2011-12-28 22:27:08,0,NaN,@TheOnion So true :),en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,@TheOnion So true :)
1,1.0,151809315026636800,151809315026636800,1325029135000.0,2011-12-27 23:38:55,0,NaN,If you ever wanted to know the *real* truth ab...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,If you ever wanted to know the *real* truth ab...
2,2.0,151338939389706242,151338939389706242,1324916990000.0,2011-12-26 16:29:50,0,NaN,Walked around a neighborhood recently rebuilt ...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walked around a neighborhood recently rebuilt ...
3,3.0,151337237429239808,151337237429239808,1324916584000.0,2011-12-26 16:23:04,0,NaN,"It was Xmas, so we brought presents for the ki...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"It was Xmas, so we brought presents for the ki..."


In [ ]:
df = df.reset_index(drop=True)
df.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,retweet_date,translate,trans_src,trans_dest,time,mentions,replies_count,retweets_count,likes_count,tidy_tweets
0,0.0,15434727182,15434727182,1275676317000.0,2010-06-04 18:31:57,0,NaN,"Please ignore prior tweets, as that was someon...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Please ignore prior tweets, as that was someon..."
1,0.0,152153637639028736,152151847614943233,1325111228000.0,2011-12-28 22:27:08,0,NaN,@TheOnion So true :),en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,@TheOnion So true :)
2,1.0,151809315026636800,151809315026636800,1325029135000.0,2011-12-27 23:38:55,0,NaN,If you ever wanted to know the *real* truth ab...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,If you ever wanted to know the *real* truth ab...
3,2.0,151338939389706242,151338939389706242,1324916990000.0,2011-12-26 16:29:50,0,NaN,Walked around a neighborhood recently rebuilt ...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walked around a neighborhood recently rebuilt ...
4,3.0,151337237429239808,151337237429239808,1324916584000.0,2011-12-26 16:23:04,0,NaN,"It was Xmas, so we brought presents for the ki...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"It was Xmas, so we brought presents for the ki..."


In [ ]:
df['absolute_tidy_tweets'] = df['tidy_tweets'].str.replace("[^a-zA-Z# ]", "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
stopwords_set = set(stopwords)
cleaned_tweets = []

for index, row in df.iterrows():
    
     words_without_stopwords = [word for word in row.absolute_tidy_tweets.split() if not word in stopwords_set and '#' not in word.lower()]
     cleaned_tweets.append(' '.join(words_without_stopwords))
    
df['absolute_tidy_tweets'] = cleaned_tweets
df.head(10)

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,translate,trans_src,trans_dest,time,mentions,replies_count,retweets_count,likes_count,tidy_tweets,absolute_tidy_tweets
0,0.0,15434727182,15434727182,1275676317000.0,2010-06-04 18:31:57,0,NaN,"Please ignore prior tweets, as that was someon...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Please ignore prior tweets, as that was someon...",Please ignore prior tweets someone pretending ...
1,0.0,152153637639028736,152151847614943233,1325111228000.0,2011-12-28 22:27:08,0,NaN,@TheOnion So true :),en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,@TheOnion So true :),TheOnion So true
2,1.0,151809315026636800,151809315026636800,1325029135000.0,2011-12-27 23:38:55,0,NaN,If you ever wanted to know the *real* truth ab...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,If you ever wanted to know the *real* truth ab...,If ever wanted know real truth moon landings b...
3,2.0,151338939389706242,151338939389706242,1324916990000.0,2011-12-26 16:29:50,0,NaN,Walked around a neighborhood recently rebuilt ...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walked around a neighborhood recently rebuilt ...,Walked around neighborhood recently rebuilt he...
4,3.0,151337237429239808,151337237429239808,1324916584000.0,2011-12-26 16:23:04,0,NaN,"It was Xmas, so we brought presents for the ki...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"It was Xmas, so we brought presents for the ki...",It Xmas brought presents kids orphanage They d...
5,4.0,151327734843445249,151327734843445249,1324914318000.0,2011-12-26 15:45:18,0,NaN,"Met with UNICEF, Doctors Without Borders and A...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Met with UNICEF, Doctors Without Borders and A...",Met UNICEF Doctors Without Borders Artists Pea...
6,5.0,151322293174419456,151322293174419456,1324913020000.0,2011-12-26 15:23:40,0,NaN,Just returned from a trip to Haiti. Covered a ...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Just returned from a trip to Haiti. Covered a ...,Just returned trip Haiti Covered lot ground sa...
7,6.0,151317672217419777,151317672217419777,1324911919000.0,2011-12-26 15:05:19,0,NaN,Single character Tweets are the ulitmate exten...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single character Tweets are the ulitmate exten...,Single character Tweets ulitmate extension Twi...
8,7.0,151151777662779392,151151777662779392,1324872366000.0,2011-12-26 04:06:06,0,NaN,I,und,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I,I
9,8.0,150390624552615937,150390624552615937,1324690893000.0,2011-12-24 01:41:33,0,NaN,The Russians are having some challenges with t...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Theussians are having some challenges with the...,Theussians challenges rockets Too many enginee...


In [ ]:
tokenized_tweet = df['absolute_tidy_tweets'].apply(lambda x: x.split())
tokenized_tweet.head()

0    [Please, ignore, prior, tweets, someone, prete...
1                                 [TheOnion, So, true]
2    [If, ever, wanted, know, real, truth, moon, la...
3    [Walked, around, neighborhood, recently, rebui...
4    [It, Xmas, brought, presents, kids, orphanage,...
Name: absolute_tidy_tweets, dtype: object

In [ ]:
import nltk
nltk.download('omw-1.4')
  

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
word_lemmatizer = WordNetLemmatizer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [word_lemmatizer.lemmatize(i) for i in x])
tokenized_tweet.head()

0    [Please, ignore, prior, tweet, someone, preten...
1                                 [TheOnion, So, true]
2    [If, ever, wanted, know, real, truth, moon, la...
3    [Walked, around, neighborhood, recently, rebui...
4    [It, Xmas, brought, present, kid, orphanage, T...
Name: absolute_tidy_tweets, dtype: object

In [ ]:
for i, tokens in enumerate(tokenized_tweet):
    tokenized_tweet[i] = ' '.join(tokens)

df['absolute_tidy_tweets'] = tokenized_tweet
df.head(10)

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,translate,trans_src,trans_dest,time,mentions,replies_count,retweets_count,likes_count,tidy_tweets,absolute_tidy_tweets
0,0.0,15434727182,15434727182,1275676317000.0,2010-06-04 18:31:57,0,NaN,"Please ignore prior tweets, as that was someon...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Please ignore prior tweets, as that was someon...",Please ignore prior tweet someone pretending T...
1,0.0,152153637639028736,152151847614943233,1325111228000.0,2011-12-28 22:27:08,0,NaN,@TheOnion So true :),en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,@TheOnion So true :),TheOnion So true
2,1.0,151809315026636800,151809315026636800,1325029135000.0,2011-12-27 23:38:55,0,NaN,If you ever wanted to know the *real* truth ab...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,If you ever wanted to know the *real* truth ab...,If ever wanted know real truth moon landing be...
3,2.0,151338939389706242,151338939389706242,1324916990000.0,2011-12-26 16:29:50,0,NaN,Walked around a neighborhood recently rebuilt ...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walked around a neighborhood recently rebuilt ...,Walked around neighborhood recently rebuilt he...
4,3.0,151337237429239808,151337237429239808,1324916584000.0,2011-12-26 16:23:04,0,NaN,"It was Xmas, so we brought presents for the ki...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"It was Xmas, so we brought presents for the ki...",It Xmas brought present kid orphanage They don...
5,4.0,151327734843445249,151327734843445249,1324914318000.0,2011-12-26 15:45:18,0,NaN,"Met with UNICEF, Doctors Without Borders and A...",en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Met with UNICEF, Doctors Without Borders and A...",Met UNICEF Doctors Without Borders Artists Pea...
6,5.0,151322293174419456,151322293174419456,1324913020000.0,2011-12-26 15:23:40,0,NaN,Just returned from a trip to Haiti. Covered a ...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Just returned from a trip to Haiti. Covered a ...,Just returned trip Haiti Covered lot ground sa...
7,6.0,151317672217419777,151317672217419777,1324911919000.0,2011-12-26 15:05:19,0,NaN,Single character Tweets are the ulitmate exten...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single character Tweets are the ulitmate exten...,Single character Tweets ulitmate extension Twi...
8,7.0,151151777662779392,151151777662779392,1324872366000.0,2011-12-26 04:06:06,0,NaN,I,und,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I,I
9,8.0,150390624552615937,150390624552615937,1324690893000.0,2011-12-24 01:41:33,0,NaN,The Russians are having some challenges with t...,en,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Theussians are having some challenges with the...,Theussians challenge rocket Too many engineer ...


In [ ]:
class PhraseExtractHelper(object):
    def __init__(self):
        self.lemmatizer = nltk.WordNetLemmatizer()
        self.stemmer = nltk.stem.porter.PorterStemmer()
    
    def leaves(self, tree):
         for subtree in tree.subtrees(filter = lambda t: t.label()=='NP'):
            yield subtree.leaves()

    def normalise(self, word):
         word = word.lower()
         word = self.lemmatizer.lemmatize(word)
         return word

    def acceptable_word(self, word):
        accepted = bool(3 <= len(word) <= 40
         and word.lower() not in stopwords
         and 'https' not in word.lower()
         and 'http' not in word.lower()
         and '#' not in word.lower()
            )
        return accepted

    def get_terms(self, tree):
        for leaf in self.leaves(tree):
            term = [ self.normalise(w) for w,t in leaf if self.acceptable_word(w) ]
            yield term

In [ ]:
sentence_re = r'(?:(?:[A-Z])(?:.[A-Z])+.?)|(?:\w+(?:-\w+)*)|(?:\$?\d+(?:.\d+)?%?)|(?:...|)(?:[][.,;"\'?():-_`])'
grammar = r"""
    NBAR:
        {<NN.*|JJ>*<NN.*>}  
        
    NP:
        {<NBAR>}
        {<NBAR><IN><NBAR>}  
"""
chunker = nltk.RegexpParser(grammar)

In [ ]:
key_phrases = []
phrase_extract_helper = PhraseExtractHelper()

for index, row in df.iterrows(): 
    toks = nltk.regexp_tokenize(row.tidy_tweets, sentence_re)
    postoks = nltk.tag.pos_tag(toks)
    tree = chunker.parse(postoks)

    terms = phrase_extract_helper.get_terms(tree)
    tweet_phrases = []

    for term in terms:
        if len(term):
            tweet_phrases.append(' '.join(term))
    
    key_phrases.append(tweet_phrases)
    
key_phrases[:10]

[['please', 'prior tweet', 'someone pretending'],
 ['theonion'],
 ['real truth', 'moon landing  ...', 'onion article'],
 ['neighborhood', 'help', ' apj', 'others'],
 ['xmas', 'present', 'kid', 'orphanage'],
 ['met',
  ' uni cef, doctor',
  'border',
  'artist',
  'peace  & j ustice',
  'others'],
 ['trip', 'haiti', 'lot', 'ground', 'many tough situation', 'lot', 'help'],
 ['single character tweet', 'ulitmate extension', 'twitmeme'],
 [],
 ['theussians', 'challenge', 'rocket', 'engineer']]

In [ ]:
target_variable = df['tweet'].apply(lambda x: 0 if x=='neg' else 1)

In [ ]:
def naive_model(X_train, X_test, y_train, y_test):
    naive_classifier = GaussianNB()
    naive_classifier.fit(X_train.toarray(), y_train)

    predictions = naive_classifier.predict(X_test.toarray())

    
    print(f'Accuracy Score - {accuracy_score(y_test, predictions)}')
    

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_word_feature, target_variable, test_size=0.3, random_state=272)
naive_model(X_train, X_test, y_train, y_test)

Accuracy Score - 1.0


In [ ]:
df['tweet'][2]

'If you ever wanted to know the *real* truth about the moon landings ...(best Onion article ever)  http://t.co/pgNEJsjI'